In [86]:
import numpy as np
import pandas as pd
import datetime
from time import time
import math
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from calendar import monthrange

In [104]:
dam_hourly = pd.read_csv('../output/dam_hourly.csv')
dam_hourly=dam_hourly.set_index(['month','hour'])
#dam_hourly=dam_hourly.drop(['date'],1)
dam_hourly

date   price  volume  year  day  day_of_week      value  \
month hour                                                                  
1     0     2016-01-01   40.10   477.5  2016    1            4  19147.750   
      1     2016-01-01   40.08   486.6  2016    1            4  19502.928   
      2     2016-01-01   40.06   495.6  2016    1            4  19853.736   
      3     2016-01-01   40.08   496.6  2016    1            4  19903.728   
      4     2016-01-01   40.07   502.9  2016    1            4  20151.203   
...                ...     ...     ...   ...  ...          ...        ...   
8     19    2021-08-31  254.20   185.0  2021   31            1  47027.000   
      20    2021-08-31  194.10   206.0  2021   31            1  39984.600   
      21    2021-08-31   99.83   213.0  2021   31            1  21263.790   
      22    2021-08-31   55.07   200.0  2021   31            1  11014.000   
      23    2021-08-31   45.09   266.0  2021   31            1  11993.940   

                 cat  
month hour            
1     0          off  
      1          off  
      2          off  
      3          off  
      4          off  
...              ...  
8     19     evening  
      20    standard  
      21    standard  
      22         off  
      23         off  

[49016 rows x 8 columns]

In [105]:
pv_p50 = pd.read_csv('../output/pv/25mw_p50.csv').melt(id_vars='Month', var_name='hour', value_name='yield').rename(columns={"Month": "month"})
pv_p50=pv_p50.astype({'hour': 'int32'})
pv_p50 = pv_p50.set_index(['month','hour'])
pv_p50

,,yield
month,hour,
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
5,0,0.0
...,...,...
8,23,0.0
9,23,0.0
10,23,0.0


In [106]:
dam_hourly['month_yield']=pv_p50['yield']
dam_hourly=dam_hourly.reset_index()
dam_hourly['month_days']=dam_hourly.apply(lambda x: monthrange(2020, x['month'])[1], axis=1)
dam_hourly['yield']=dam_hourly['month_yield']/dam_hourly['month_days']
dam_hourly

,month,hour,date,price,volume,year,day,day_of_week,value,cat,month_yield,month_days,yield
0,1,0,2016-01-01,40.10,477.5,2016,1,4,19147.750,off,0.0,31,0.0
1,1,1,2016-01-01,40.08,486.6,2016,1,4,19502.928,off,0.0,31,0.0
2,1,2,2016-01-01,40.06,495.6,2016,1,4,19853.736,off,0.0,31,0.0
3,1,3,2016-01-01,40.08,496.6,2016,1,4,19903.728,off,0.0,31,0.0
4,1,4,2016-01-01,40.07,502.9,2016,1,4,20151.203,off,0.0,31,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49011,8,19,2021-08-31,254.20,185.0,2021,31,1,47027.000,evening,0.0,31,0.0
49012,8,20,2021-08-31,194.10,206.0,2021,31,1,39984.600,standard,0.0,31,0.0
49013,8,21,2021-08-31,99.83,213.0,2021,31,1,21263.790,standard,0.0,31,0.0
49014,8,22,2021-08-31,55.07,200.0,2021,31,1,11014.000,off,0.0,31,0.0
